In [1]:
!pip install langchain-groq
!pip install langchain-community
!pip install pdfplumber

In [2]:
import os
from langchain_groq import ChatGroq
from langchain_community.document_loaders import PyPDFLoader
from langchain.chains.summarize import load_summarize_chain
from langchain.chains import ConversationalRetrievalChain
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
import gdown
import json

In [4]:
os.environ["GROQ_API_KEY"] = "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"
model = ChatGroq(temperature=0.8, model="llama3-8b-8192")

In [6]:
!pip install pypdf

In [7]:
def load_pdf_text(pdf_path):
    loader = PyPDFLoader(pdf_path)
    pages = loader.load_and_split()
    return [page.page_content for page in pages]
pdf_path = "/content/BDM Capstone Project Document for 2024.pdf"
document_texts = load_pdf_text(pdf_path)
embedder = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vector_store = FAISS.from_texts(document_texts, embedder)

<ipython-input-7-e45a84d69a8c>:12: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedder = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token

In [8]:
retrieval_chain = ConversationalRetrievalChain.from_llm(model, retriever=vector_store.as_retriever())


In [9]:
print("Chatbot is ready! Ask questions about the PDF content, or type 'stop' to exit.")
chat_history = []

while True:
    user_input = input("You: ")
    if user_input.lower() == 'stop':
        print("Chatbot: Goodbye!")
        break
    response = retrieval_chain.invoke({"question": user_input, "chat_history": chat_history})
    chat_history.append((user_input, response["answer"]))
    answer = response["answer"]
    formatted_answer = "\n".join(" ".join(answer.split()[i:i+15]) for i in range(0, len(answer.split()), 15))
    print("Chatbot: ", formatted_answer)

Chatbot is ready! Ask questions about the PDF content, or type 'stop' to exit.
You: who wrote this document
Chatbot:
 This document was written jointly by Dr. Aaditya Chandel and Dr. Ashwin J. Baliga.
You: when to submit
Chatbot:
 I don't know the answer to that. The provided document does not mention a submission
deadline.
You: how many days to submit
Chatbot:
 I don't know. The provided document does not mention a specific submission deadline for the
document.
You: in how many months i have to complete the BDM project
Chatbot:
 The document does not specify the exact number of months you have to complete the
BDM project. It only mentions that the project is not just about showing data and
graphs, and that you should ensure that your report is written well and does not
mix active and passive voice. It also recommends that you read the reference books shared
and go through all the live sessions, FAQs, documents, and the reference books to facilitate
better understanding.
You: give me s